In [ ]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json backoff xxhash pyyaml

In [ ]:
! pip install --quiet git+https://github.com/nandanrao/facebook-python-business-sdk@pagination

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from environs import Env
env = Env()
env.read_env('.env-bug')

In [5]:
from malaria import get_conf, get_df, lookalike
from marketing import Marketing

cnf = get_conf(env)
# m = Marketing(env)

In [ ]:
from malaria import update_audience, update_ads

# update_audience()

update_ads()

In [6]:
from datetime import datetime, timezone
from clustering import only_target_users

def filter_time(df, min_date, max_date):
    min_date = datetime(*min_date, tzinfo=timezone.utc)
    max_date = datetime(*max_date, tzinfo=timezone.utc)

    users = df[(df.timestamp > min_date) & (df.timestamp < max_date)].userid.unique()
    return df[df.userid.isin(users)]

def get_percentage(df, cnf):
    answered = df[df.question_ref == 'f6e69027-97cc-494e-8d52-318b75047e23'].userid.unique()
    df = df[df.userid.isin(answered)]
    tot = df.userid.unique().shape[0]
    targets = only_target_users(df, cnf['strata'][0]['surveys'], 'target_questions')
    targets = targets.userid.unique().shape[0]
    return targets/tot, targets, tot


In [7]:
import pandas as pd
from clustering import add_res_cols
from malaria import load_cities


def perc_target(users, df):
    u = pd.Series(df.userid.unique())
    tot = u.unique().shape[0]
    target = u.isin(users).sum()
    perc = target / tot
    return pd.Series([perc, tot, target], index=['perc', 'tot', 'target'])


def make_analysis_df(df, districts, users):
    finished = df[df.question_ref == 'f6e69027-97cc-494e-8d52-318b75047e23'].userid.unique()
    df = df[df.userid.isin(finished)].reset_index(drop=True)
    df = add_res_cols('md:clusterid', 'cluster', 'userid', df)
    perc = df.groupby('cluster').apply(lambda df: perc_target(users, df)).reset_index()
    perc = perc.merge(districts, left_on='cluster', right_on='disthash')
    return perc

In [9]:

cities = load_cities('output/cities.csv')

districts = cities.groupby('distname') \
                  .head(1) \
                  .reset_index(drop=True) \
                  .drop(columns=['rad', 'lng', 'lat', 'distcode', 'id'])


# perc = make_analysis_df(df, districts, users)

In [ ]:
def statewide(df):
    trg = df.target.sum().astype(int)
    tot = df.tot.sum().astype(int)
    return pd.Series([trg/tot, trg, tot], index=['percentage', 'malaria', 'total'])

states = perc.groupby('state').apply(statewide).round(4)
states['total'] = states['total'].astype(int)
states['malaria'] = states['malaria'].astype(int)

states

In [122]:
chosen_clusters = perc[(perc.target >= 3) & (perc.perc > 0.1)].cluster
top_cities = cities[cities.disthash.isin(chosen_clusters)].reset_index(drop=True)

In [ ]:
perc[perc.perc > .35].sort_values('target')

In [90]:
chosen_clusters = perc[(perc.perc > .35) & (perc.target >= 3)].cluster
malaria_cities = cities[cities.disthash.isin(chosen_clusters)].reset_index(drop=True)
malaria_cities = malaria_cities[~malaria_cities.state.isin(['Orissa'])]
malaria_cities.to_csv('output/malaria-cities.csv', index=False)

In [ ]:
malaria_cities

In [94]:
import pandas as pd

top_cities = pd.read_csv('output/top-cities.csv')

In [ ]:
top_cities

In [26]:

def pause_adsets(m, clusters):
    get_ad = lambda c: next((a for k, a in m.running_ads.items() if c in k), None)
    to_delete = [get_ad(c) for c in clusters]
    upd = lambda a: a.api_update(params={ 'status': 'PAUSED' })
    return [upd(ad) for ad in to_delete]
        
# res = pause_adsets(m, perc[perc.perc == 0.0].cluster)

In [ ]:
aud = get_aud(m, cnf, True)

In [ ]:
aud

In [9]:
from malaria import unsaturated

clusters = unsaturated(df, cnf)

In [75]:
adsets = m.account.get_ad_sets(fields=['lifetime_budget', 'instagram_actor_id', 'campaign_id', 'targeting'])

In [55]:
import hashlib
import hmac

def secret_proof(app_secret, access_token):
    h = hmac.new (
        app_secret.encode('utf-8'),
        msg=access_token.encode('utf-8'),
        digestmod=hashlib.sha256
    )
    return h.hexdigest()


secret = env('FACEBOOK_APP_SECRET')
token = ''
user = users[0]

proof = secret_proof(secret, token)

import requests

res = requests.get(f'https://graph.facebook.com/v7.0/{user}/ids_for_apps?access_token={token}&appsecret_proof={proof}')

res.text

'{"error":{"message":"(#100) An owning business is required for this request","type":"OAuthException","code":100,"fbtrace_id":"AypFAF9YX6tqhTqMedHqQHB"}}'

In [33]:
import time

from facebook_business.adobjects.serverside.event import Event
from facebook_business.adobjects.serverside.event_request import EventRequest
from facebook_business.adobjects.serverside.user_data import UserData
from facebook_business.adobjects.serverside.custom_data import CustomData
from facebook_business.api import FacebookAdsApi

# access_token = '<ACCESS_TOKEN>'
# pixel_id = 'ADS_PIXEL_ID>'


# user_data = UserData(
#     email='joe@eg.com',
#     fbc='fb.1.1554763741205.AbCdEfGhIjKlMnOpQrStUvWxYz1234567890',
#     fbp='fb.1.1558571054389.1098115397'
# )

# custom_data = CustomData(
#     currency='usd',
#     value=123.45
# )

# event = Event(
#     event_name='Purchase',
#     event_time=int(time.time()),
#     user_data=user_data,
#     custom_data=custom_data,
# )

# events = [event]

# event_request = EventRequest(
#     events=events,
#     pixel_id=pixel_id)

# event_response = event_request.execute()
# print(event_response)

In [6]:

cities = load_cities('output/cities.csv')



In [ ]:
from marketing import get_creatives, get_label
from facebook_business.adobjects.adcreative import AdCreative

def delete_our_creatives(account, label):
    label = get_label(account, label)
    creatives = get_creatives(account, label['id'])
    print(len(creatives))

    i = 0
    for c in creatives.values():
        try:
            c.api_delete()
            i += 1
            print(i)
        except Exception as e:
            print(e)

# delete_our_creatives(m.account, 'virtuallab-1')

In [10]:
from responses import get_forms
from datetime import datetime

survey_user = '10383123-9fb1-429b-8312-447c1b54b17a'
shortcodes = ['baselinehin', 'baselineeng']

timestamp = datetime(2020, 7, 25)

forms = list(get_forms(survey_user, shortcodes, timestamp, cnf['chatbase']))

In [11]:
questions = [
    ('malaria_incidence', 'f6e69027-97cc-494e-8d52-318b75047e23'),
    ('dist_medical', 'd7573919-8a7e-457f-9a1d-1f8c389127a7'),
    ('education', 'e40fa1c6-13a1-4a02-91cd-0eaade11864d'),
    ('owns_net', 'bd4802c6-7bdb-40f0-aac1-18cc6df7da6e'),
    ('has_ac', 'e279587c-975f-433a-adab-1ad563876af6'),
    ('has_ac', 'e279587c-975f-433a-adab-1ad563876af6'),
    ('net_lastnight', 'b5b1ff58-c8df-4890-9b1c-0cd40ce6edc0'),
    ('home', '4d0ae478-3893-4b46-ab39-d6848c69245d')]


In [12]:
from responses import get_response_df

def ref_translation(eng_form, other_form):
    vals = [(a['ref'], b['ref']) for a, b 
            in zip(eng_form['fields'], other_form['fields'])]
    lookup = dict(vals)
    rev_lookup = {v:k for k,v in lookup.items()}
    return lookup, rev_lookup


def get_filtered_responses(survey_user, eng_form, other_form, questions, db_cnf):
    ref_lookup, rev_ref_lookup = ref_translation(eng_form, other_form)
    refs = [ref_lookup[r] for _, r in questions]
    responses = get_response_df(survey_user, ['baselinehin'], refs, db_cnf)

    _, last_ref = questions[-1]
    
    answered = responses[responses.question_ref == last_ref].userid.unique()    

    return responses[responses.userid.isin(answered)].reset_index(drop=True)


filtered_res = get_filtered_responses(survey_user, forms[0], forms[1], questions, cnf['chatbase'])

In [13]:
from clustering import add_res_cols
from forms import response_translator

def col_translators(a, b, questions):
    lookup, _ = ref_translation(a, b)

    refs = [ref for _, ref in questions]
    ts = { qt['ref']: response_translator(q, qt)
                   for qt, q in zip(a['fields'], b['fields'])
                   if qt['ref'] in refs}

    return [(n, lookup[ref], ts[ref]) for n, ref in questions]

col_names = col_translators(forms[0], forms[1], questions)
rr = add_res_cols(col_names, 'userid', filtered_res).reset_index(drop=True)

In [14]:
rr = add_res_cols([('cluster', 'md:clusterid', lambda x: x)], 'userid', rr).reset_index(drop=True)

In [15]:
def make_row(df):
    r = df.iloc[0]
    r.timestamp = df.timestamp.max()
    return r

dd = rr.groupby('userid').apply(make_row).reset_index(drop=True)
dd = dd.merge(districts, left_on='cluster', right_on='disthash')


In [17]:
dd['under_net'] = dd.net_lastnight == 'Yes'
dd['malaria'] = dd.malaria_incidence == 'Yes'

In [18]:
def malaria_prob(groupby):
    d = groupby.malaria.value_counts(normalize=True)
    d.name = 'count'
    d = d.reset_index(level=-1)
    d = d[d.malaria == True].drop(columns='malaria')
    return d

In [33]:
def stats(df):
    targets = df.malaria.sum()
    tot = df.shape[0]
    non_target = tot - targets
    return pd.Series([targets/tot, targets, non_target, tot], index=['perc', 'target', 'non_target', 'tot'])

perc = dd.groupby('disthash').apply(stats).reset_index()

In [20]:
to_drop = perc[(perc.tot <= 6) & (perc.perc <= 0.05)].disthash

In [27]:
cities = cities[~cities.disthash.isin(to_drop)]

In [30]:
dd = dd[~dd.disthash.isin(to_drop)].reset_index(drop=True)

In [ ]:
with pd.option_context('display.max_rows', None): 
    display(perc.sort_values('target'))

In [37]:
high_malaria_dists = perc[(perc.perc > .25)].disthash.unique()
low_malaria_dists = perc[(perc.perc <= .15) & (perc.tot >= 20)].disthash.unique()
# low_malaria_dists = perc[(perc.perc == 0.0) & (perc.target >= 0)].disthash.unique()

dd['dist_risk'] = 'med'
dd.loc[dd.cluster.isin(low_malaria_dists), 'dist_risk'] = 'low'
dd.loc[dd.cluster.isin(high_malaria_dists), 'dist_risk'] = 'high'

In [38]:
dd.groupby('disthash').head(1)['dist_risk'].value_counts()

med     45
low     32
high    32
Name: dist_risk, dtype: int64

In [48]:
dd['include_audience'] = dd.dist_risk == 'low'

cities = cities.merge(dd.groupby('disthash').head(1).reset_index(drop=True)[['disthash', 'include_audience']])

cities.to_csv('output/cities.csv', index=False)

In [53]:
filter_time(dd, (2020,7,21), (2020,7,25)) \
    .pipe(lambda df: df[df.disthash.isin(low_malaria_dists)]).shape[0] / filter_time(dd, (2020,7,21), (2020,7,24)).shape[0]

0.2022106631989597

In [54]:
filter_time(dd, (2020,8,1), (2020,8,10)) \
    .pipe(lambda df: df[df.disthash.isin(low_malaria_dists)]).shape[0] / filter_time(dd, (2020,7,21), (2020,7,24)).shape[0]

0.016905071521456438

In [55]:
filter_time(dd[dd.disthash.isin(low_malaria_dists)], (2020,7,21), (2020,7,24)).home.value_counts(normalize=True)

Pucca (have cement/brick wall and floor    0.626050
Semi-pucca                                 0.252101
Kutcha (made of mud, tin, straw)           0.121849
Name: home, dtype: float64

In [56]:
filter_time(dd[dd.disthash.isin(low_malaria_dists)], (2020,8,1), (2020,8,10)).home.value_counts(normalize=True)

Pucca (have cement/brick wall and floor    0.615385
Kutcha (made of mud, tin, straw)           0.230769
Semi-pucca                                 0.153846
Name: home, dtype: float64

In [72]:
saturated = perc[perc.non_target > 150].disthash

56    786f66b7
Name: disthash, dtype: object

In [79]:
cities['creative_group'] = 'hindi'
cities.loc[cities.disthash.isin(saturated), 'creative_group'] = 'exclusions'

In [84]:
cities.to_csv('output/cities.csv', index=False)

In [8]:
import pandas as pd

cities = pd.read_csv('output/cities.csv')

In [ ]:
for (i,n,cg), df in cities.groupby(['disthash', 'distname', 'creative_group']):
    print(cg)

In [ ]:
malaria_prob(dd.groupby('under_net')) 

In [ ]:
dd.groupby('dist_risk').home.value_counts(normalize=True)

In [ ]:
# district fixed effects of puccaness

dd['pucca'] = dd.home != 'Kutcha (made of mud, tin, straw)'

malaria_prob(dd.groupby(['disthash', 'pucca'])).reset_index() \
    .groupby('disthash') \
    .filter(lambda df: df.shape[0] > 1) \
    .groupby('disthash') \
    .apply(lambda df: df.iloc[0]['count'] - df.iloc[1]['count']).quantile(0.5)

In [ ]:
malaria_prob(filter_time(dd, (2020,7,21), (2020,7,26)).groupby(['dist_risk', 'home']))

In [63]:
malaria_prob(filter_time(dd, (2020,8,1), (2020,9,1)).groupby(['dist_risk', 'home']))

count
dist_risk home                                             
high      Kutcha (made of mud, tin, straw)         0.350000
          Pucca (have cement/brick wall and floor  0.238095
          Semi-pucca                               0.375000
med       Kutcha (made of mud, tin, straw)         0.187500
          Pucca (have cement/brick wall and floor  0.151899
          Semi-pucca                               0.177778

In [ ]:
filter_time(dd, (2020,8,1), (2020,9,1)).pipe(lambda df: df[df.dist_risk == 'low'])

In [550]:
malaria_prob(dd.groupby(['has_ac']))

,count
has_ac,
Don't know,0.141463
No,0.199307
Yes,0.156977


In [85]:
malaria_prob(dd.groupby(['dist_risk', 'home', 'has_ac']))

count
dist_risk home                                    has_ac              
high      Kutcha (made of mud, tin, straw)        Don't know  0.304348
                                                  No          0.425926
                                                  Yes         0.500000
          Pucca (have cement/brick wall and floor Don't know  0.240000
                                                  No          0.366477
                                                  Yes         0.283333
          Semi-pucca                              Don't know  0.181818
                                                  No          0.424084
                                                  Yes         0.285714
low       Kutcha (made of mud, tin, straw)        No          0.142857
          Pucca (have cement/brick wall and floor No          0.034483
          Semi-pucca                              No          0.066667
med       Kutcha (made of mud, tin, straw)        Don't know  0.103448
                                                  No          0.183365
                                                  Yes         0.230769
          Pucca (have cement/brick wall and floor Don't know  0.126761
                                                  No          0.141593
                                                  Yes         0.134286
          Semi-pucca                              Don't know  0.116279
                                                  No          0.176179
                                                  Yes         0.150943

In [531]:
malaria_prob(dd.groupby(['owns_net']))

,count
owns_net,
Don't know,0.066667
No,0.154300
Yes,0.207216


In [549]:
malaria_prob(dd.groupby(['dist_risk', 'owns_net']))

count
dist_risk owns_net            
high      Don't know  0.166667
          No          0.338462
          Yes         0.437055
low       No          0.137500
          Yes         0.065693
med       Don't know  0.055556
          No          0.130580
          Yes         0.191207

In [ ]:
malaria_prob(dd.groupby(['state', 'dist_risk', 'owns_net']))

In [ ]:
X = dd[['education', 'home', 'dist_medical', 'has_ac', 'owns_net']]


for col in X.columns:
    X[col] = X[col].astype('category')
    X[col] = X[col].cat.codes

In [ ]:
X

In [423]:
X.shape

(4474, 5)

In [425]:
y = dd['malaria']

In [ ]:
y.shape

In [542]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

model.fit(X[:2000], y[:2000])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:

from sklearn.tree import plot_tree

plot_tree(model)

In [300]:
dd[(dd.owns_net == 'Yes') & 
   (dd.has_ac == 'No') & 
   (dd.home == 'Kutcha (made of mud, tin, straw)')] \
   .malaria_incidence.value_counts(normalize=True)

No            0.636550
Yes           0.268994
Don’t know    0.094456
Name: malaria_incidence, dtype: float64

In [ ]:
dd

In [330]:
def times(df, mi, ma):
    return df[(df.timestamp > mi) & (df.timestamp < ma)]    


times(dd, datetime(2020,7,22,tzinfo=timezone.utc), datetime(2020,7,24,tzinfo=timezone.utc)) \
    ['has_ac'] \
    .value_counts(normalize=True)

No            0.846106
Yes           0.113693
Don't know    0.040201
Name: has_ac, dtype: float64

In [331]:
times(dd, datetime(2020,7,24,tzinfo=timezone.utc), datetime(2020,7,25,tzinfo=timezone.utc)) \
    ['has_ac'] \
    .value_counts(normalize=True)

No            0.797244
Yes           0.147638
Don't know    0.055118
Name: has_ac, dtype: float64

In [332]:
times(dd, datetime(2020,7,25,tzinfo=timezone.utc), datetime(2020,7,26,tzinfo=timezone.utc)) \
    ['has_ac'] \
    .value_counts(normalize=True)

No            0.862589
Yes           0.090426
Don't know    0.046986
Name: has_ac, dtype: float64

In [328]:
times(dd, datetime(2020,7,26,tzinfo=timezone.utc), datetime(2020,7,28,tzinfo=timezone.utc)) \
    ['has_ac'] \
    .value_counts(normalize=True)

No            0.825040
Yes           0.126806
Don't know    0.048154
Name: has_ac, dtype: float64

In [ ]:
dd

In [301]:
dd.malaria_incidence.value_counts(normalize=True)

No            0.670988
Yes           0.191775
Don’t know    0.137237
Name: malaria_incidence, dtype: float64

In [149]:
# responses should be converted and translated...